In [ ]:
from utils import *
from plotly.subplots import make_subplots
from tqdm.auto import tqdm
from loaders import baseline_loader

In [ ]:
res_df, scalars = baseline_loader()

In [ ]:
from itertools import islice

envs = sorted([*res_df["env"].unique()])
ratios = sorted([*res_df["ratio"].unique()])

specs = [[{} for _ in ratios] for _ in envs]

fig = make_subplots(
    rows=len(envs),
    row_titles=[*envs],
    cols=len(ratios),
    column_titles=[f"K = {r}" for r in ratios],
    horizontal_spacing=0.025,
    vertical_spacing=0.05,
    specs=specs,
    shared_yaxes=True,
    shared_xaxes=True,
)

val_ranges = {
    "Assault": [1.5, 5.0],
    "CrazyClimber": [0.5, 1.5],
    "MsPacman": [1.2, 2.5],
}

color_iter = make_color_iter()
colors = [next(color_iter) for _ in ratios]

axis = 1
for row, env in enumerate(envs, 1):
    env_dfs = res_df[res_df["env"] == env]
    for col, ratio in enumerate(ratios, 1):
        ratio_dfs = env_dfs[env_dfs["ratio"] == ratio]

        dfs = []
        for _, test in ratio_dfs.iterrows():
            test_df = scalars.read(test["path"])
            test_df = test_df[test_df["tag"] == "wm/val_loss"].copy()
            dfs.append(test_df)
        dfs = pd.concat(dfs)
        g = dfs.groupby("step")
        df = pd.concat({"value": g["value"].mean()})
        df = df.reset_index()
        df["step"] *= ratio

        traces = [
            go.Scatter(
                x=df["step"],
                y=exp_mov_avg(df["value"], 1.0 - 5e-3 * ratio),
                showlegend=False,
                # legendgroup=f"{ratio}",
                name=f"Ratio = {ratio}",
                line=dict(color=colors[col - 1]),
            ),
        ]
        for trace in traces:
            fig.add_trace(trace, row=row, col=col)

        fig.update_layout(**{f"yaxis{axis}": dict(range=val_ranges[env])})
        axis += 1


fig.update_layout(width=1000, height=500)
fig.write_image("../tex/assets/model_val_curves.pdf")
fig

In [ ]:
envs = res_df["env"].unique()

fig = make_subplots(
    rows=1,
    cols=len(envs),
    column_titles=[*envs],
)

axis = 1
for col, env in enumerate(envs, 1):
    score_df = res_df[res_df["env"] == env].copy()

    final_values = []
    for idx, row in score_df.iterrows():
        test_df = scalars.read(row["path"])
        values = test_df[test_df["tag"] == "val/wm_loss"]["value"].to_numpy()
        # val_loss = exp_mov_avg(val_loss, 0.9)
        final_values.append(values[-1])

    score_df["wm_loss"] = final_values

    subplot = px.scatter(score_df, x="score", y="wm_loss", trendline="ols")
    for trace in subplot.data:
        fig.add_trace(trace, row=1, col=col)

    fig.update_layout(**{f"xaxis{axis}": dict(title="Score")})
    if col == 1:
        fig.update_layout(**{f"yaxis{axis}": dict(title="Model val loss")})
    axis += 1

fig.update_layout(width=1000, height=400)

fig.write_image("../tex/assets/model_val_loss.pdf")
fig

In [ ]:
from itertools import islice

envs = sorted([*res_df["env"].unique()])
ratios = sorted([*res_df["ratio"].unique()])

specs = [[{} for _ in ratios] for _ in envs]

fig = make_subplots(
    rows=len(envs),
    row_titles=[*envs],
    cols=len(ratios),
    column_titles=[str(r) for r in ratios],
    shared_yaxes=True,
    shared_xaxes=True,
    horizontal_spacing=0.025,
    vertical_spacing=0.05,
    specs=specs,
)

val_ranges = {
    "Assault": [0.2, 5.0],
    "CrazyClimber": [0.075, 1.0],
    "MsPacman": [0.15, 2.0],
}

color_iter = make_color_iter()
colors = [next(color_iter) for _ in ratios]

axis = 1
for row, env in enumerate(envs, 1):
    env_dfs = res_df[res_df["env"] == env]
    for col, ratio in enumerate(ratios, 1):
        ratio_dfs = env_dfs[env_dfs["ratio"] == ratio]

        dfs = []
        for _, test in ratio_dfs.iterrows():
            test_df = scalars.read(test["path"])
            test_df = test_df[test_df["tag"] == "wm/loss"].copy()
            dfs.append(test_df)
        dfs = pd.concat(dfs)
        g = dfs.groupby("step")
        df = pd.concat({"value": g["value"].mean()})
        df = df.reset_index()
        df["step"] *= ratio

        traces = [
            go.Scatter(
                x=df["step"],
                y=exp_mov_avg(df["value"], 1.0 - 5e-3 * ratio),
                showlegend=False,
                # legendgroup=f"{ratio}",
                name=f"Ratio = {ratio}",
                line=dict(color=colors[col - 1]),
            ),
            # go.Scatter(
            #     x=df["step"],
            #     y=df["value"],
            #     showlegend=False,
            #     legendgroup=f"{ratio}",
            #     name=f"Ratio = {ratio}",
            #     line=dict(color=to_rgba(colors[col - 1], 0.2)),
            # ),
        ]
        for trace in traces:
            fig.add_trace(trace, row=row, col=col)

        yaxis_name = f"yaxis{axis if axis > 1 else ''}"
        min_, max_ = val_ranges[env]
        fig.update_layout(
            **{
                yaxis_name: dict(
                    type="log",
                    range=[np.log10(min_), np.log10(max_)],
                )
            }
        )
        axis += 1


fig.update_layout(width=1000, height=500)
fig.write_image("../tex/assets/model_train_curves.pdf")
fig

In [ ]:
envs = res_df["env"].unique()

fig = make_subplots(
    rows=1,
    cols=len(envs),
    column_titles=[*envs],
)

axis = 1
for col, env in enumerate(envs, 1):
    score_df = res_df[res_df["env"] == env].copy()

    final_values = []
    for idx, row in score_df.iterrows():
        test_df = scalars.read(row["path"])
        values = test_df[test_df["tag"] == "wm/loss"]["value"].to_numpy()
        # val_loss = exp_mov_avg(val_loss, 0.9)
        final_values.append(values[-1])

    score_df["wm_loss"] = final_values

    subplot = px.scatter(score_df, x="score", y="wm_loss", trendline="ols")
    for trace in subplot.data:
        fig.add_trace(trace, row=1, col=col)

    fig.update_layout(**{f"xaxis{axis}": dict(title="Score")})
    if col == 1:
        fig.update_layout(**{f"yaxis{axis}": dict(title="Model train loss")})
    axis += 1

fig.update_layout(width=1000, height=400)

fig.write_image("../tex/assets/model_train_loss.pdf")
fig

In [ ]:
from itertools import islice

envs = sorted([*res_df["env"].unique()])
ratios = sorted([*res_df["ratio"].unique()])

specs = [[dict(secondary_y=True) for _ in ratios] for _ in envs]

fig = make_subplots(
    rows=len(envs),
    row_titles=[*envs],
    cols=len(ratios),
    column_titles=[f"K = {r}" for r in ratios],
    horizontal_spacing=0.025,
    vertical_spacing=0.05,
    specs=specs,
    shared_yaxes=True,
    shared_xaxes=True,
)

val_ranges = {
    "Assault": [1.5, 5.0],
    "CrazyClimber": [0.5, 1.5],
    "MsPacman": [1.2, 2.5],
}

ret_ranges = {
    "Assault": [None, 800],
    "CrazyClimber": [None, 60e3],
    "MsPacman": [None, 1400],
}

color_iter = make_color_iter()
colors = [next(color_iter) for _ in range(len(ratios) * 2)]

axis = 1
for row, env in enumerate(envs, 1):
    env_dfs = res_df[res_df["env"] == env]
    for col, ratio in enumerate(ratios, 1):
        ratio_dfs = env_dfs[env_dfs["ratio"] == ratio]

        val_rets_t = np.arange(int(20e3), int(400e3 + 1), int(20e3))
        val_loss_t = np.arange(int(20e3), int(400e3 + 1), 16)
        all_val_rets, all_val_loss = [], []
        for _, res_row in ratio_dfs.iterrows():
            df = scalars.read(res_row["path"])
            val_rets = df[df["tag"] == "val/mean_ep_ret"]
            val_rets_v = np.interp(val_rets_t, val_rets["step"], val_rets["value"])
            all_val_rets.append(val_rets_v)
            val_loss = df[df["tag"] == "wm/val_loss"]
            wm_step_map = df[df["tag"] == "wm/env_step"]
            val_loss_e = np.interp(
                val_loss["step"], wm_step_map["step"], wm_step_map["value"]
            )
            val_loss_v = np.interp(val_loss_t, val_loss_e, val_loss["value"])
            val_loss_v = exp_mov_avg(val_loss_v, 0.975)
            all_val_loss.append(val_loss_v)

        all_val_rets, all_val_loss = np.stack(all_val_rets), np.stack(all_val_loss)

        for trace in err_line(
            val_loss_t,
            all_val_loss.mean(0),
            all_val_loss.std(0),
            colors[0],
            showlegend=False,
        ):
            fig.add_trace(trace, row=row, col=col, secondary_y=False)

        for trace in err_line(
            val_rets_t,
            all_val_rets.mean(0),
            all_val_rets.std(0),
            colors[1],
            showlegend=False,
        ):
            fig.add_trace(trace, row=row, col=col, secondary_y=True)

        # fig.add_trace(
        #     go.Scatter(
        #         x=val_loss_t,
        #         y=all_val_loss.mean(axis=0),
        #         showlegend=False,
        #         line=dict(color=colors[2 * (col - 1)]),
        #     ),
        #     row=row,
        #     col=col,
        #     secondary_y=False,
        # )

        # fig.add_trace(
        #     go.Scatter(
        #         x=val_rets_t,
        #         y=all_val_rets.mean(axis=0),
        #         showlegend=False,
        #         line=dict(color=colors[2 * (col - 1) + 1]),
        #     ),
        #     row=row,
        #     col=col,
        #     secondary_y=True,
        # )

        fig.update_yaxes(
            range=val_ranges[env],
            secondary_y=False,
            row=row,
            col=col,
        )

        fig.update_yaxes(
            range=ret_ranges[env],
            secondary_y=True,
            row=row,
            col=col,
        )

        if col > 1:
            fig.update_yaxes(
                visible=False,
                secondary_y=True,
                row=row,
                col=col,
            )


fig.update_layout(width=1000, height=500)
fig.write_image("../tex/assets/val_loss_v_perf.pdf")
fig

In [ ]:
from itertools import islice

envs = sorted([*res_df["env"].unique()])
ratios = sorted([*res_df["ratio"].unique()])

specs = [[dict(secondary_y=True) for _ in ratios] for _ in envs]

fig = make_subplots(
    rows=len(envs),
    row_titles=[*envs],
    cols=len(ratios),
    column_titles=[f"K = {r}" for r in ratios],
    horizontal_spacing=0.025,
    vertical_spacing=0.05,
    specs=specs,
    shared_yaxes=True,
    shared_xaxes=True,
)

train_ranges = {
    "Assault": [0.2, 5.0],
    "CrazyClimber": [0.075, 1.0],
    "MsPacman": [0.15, 2.0],
}

ret_ranges = {
    "Assault": [None, 800],
    "CrazyClimber": [None, 60e3],
    "MsPacman": [None, 1400],
}

color_iter = make_color_iter()
colors = [next(color_iter) for _ in range(len(ratios) * 2)]

axis = 1
for row, env in enumerate(envs, 1):
    env_dfs = res_df[res_df["env"] == env]
    for col, ratio in enumerate(ratios, 1):
        ratio_dfs = env_dfs[env_dfs["ratio"] == ratio]

        val_rets_t = np.arange(int(20e3), int(400e3 + 1), int(20e3))
        train_loss_t = np.arange(int(20e3), int(400e3 + 1), 16)
        all_val_rets, all_train_loss = [], []
        for _, res_row in ratio_dfs.iterrows():
            df = scalars.read(res_row["path"])
            val_rets = df[df["tag"] == "val/mean_ep_ret"]
            val_rets_v = np.interp(val_rets_t, val_rets["step"], val_rets["value"])
            all_val_rets.append(val_rets_v)
            train_loss = df[df["tag"] == "wm/loss"]
            wm_step_map = df[df["tag"] == "wm/env_step"]
            train_loss_e = np.interp(
                train_loss["step"], wm_step_map["step"], wm_step_map["value"]
            )
            train_loss_v = np.interp(train_loss_t, train_loss_e, train_loss["value"])
            train_loss_v = exp_mov_avg(train_loss_v, 0.975)
            all_train_loss.append(train_loss_v)

        all_val_rets, all_train_loss = np.stack(all_val_rets), np.stack(all_train_loss)

        for trace in err_line(
            train_loss_t,
            all_train_loss.mean(0),
            all_train_loss.std(0),
            colors[0],
            showlegend=False,
        ):
            fig.add_trace(trace, row=row, col=col, secondary_y=False)

        for trace in err_line(
            val_rets_t,
            all_val_rets.mean(0),
            all_val_rets.std(0),
            colors[1],
            showlegend=False,
        ):
            fig.add_trace(trace, row=row, col=col, secondary_y=True)

        min_, max_ = train_ranges[env]
        fig.update_yaxes(
            type="log",
            range=[np.log10(min_), np.log10(max_)],
            secondary_y=False,
            row=row,
            col=col,
        )

        fig.update_yaxes(
            range=ret_ranges[env],
            secondary_y=True,
            row=row,
            col=col,
        )

        if col > 1:
            fig.update_yaxes(
                visible=False,
                secondary_y=True,
                row=row,
                col=col,
            )


fig.update_layout(width=1000, height=500)
fig.write_image("../tex/assets/train_loss_v_perf.pdf")
fig